In [50]:
import numpy as np
import pandas as pd
import torchvision
import torchvision.transforms as transforms
import sklearn
from PIL import Image
import torch
import os
from glob import glob

In [25]:
# set device 
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [26]:
# set param
batch_size = 256
epoch = 100
lr = 0.001

In [27]:
# set random seed
torch.manual_seed(1)
if device == "cuda":
    torch.cuda.manual_seed_all(1)

In [39]:
# set dir path
train_dir_path = "./Week10/2021-ai-w10-p2/images/images"
test_dir_path = "./Week10/2021-ai-w10-p2/test_data/test_data"
# submission = pd.read_csv("./")

In [43]:
# set custom dataset
class myDataset(torch.utils.data.Dataset):
    def __init__(self,split,path,transform=None):
        self.split = split
        self.transform = transform
        if self.split == "train":
            self.path = glob(os.path.join(path,"*","*"))
            self.path = [ x.replace("\\","/") for x in self.path]
            self.label = [ int(x.split("/")[-2]) for x in self.path]
        else:
            self.path = glob(os.path.join(path,"*"))
            self.path = [ x.replace("\\","/") for x in self.path]
            # sorted
        self.len = len(self.path)
        
    def __len__(self):
        return self.len
    
    def __getitem__(self,index):
        img = Image.open(self.path[index])
        if transform is None :
            img = np.array(img)
            img = img.reshape(256,256,3).repeat(3,-1).transpose((0,3,1,2))
            if self.split == "train":
                return torch.Tensor(img), torch.LongTensor(self.label)
            else:
                return torch.Tensor(img)
        else:
            img = self.transpoform(img)
            img = np.array(img)
            if self.split == "train":
                return torch.Tensor(img), torch.LongTensor(self.label)
            else:
                return torch.Tensor(img)

In [44]:
# definde transform
transform = transforms.Compose([transforms.Resize((256,256)),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),transforms.ToTensor()])

In [45]:
# use dataset
train_dataset = myDataset("train",train_dir_path,transform)
test_dataset = myDataset("test",test_dir_path,transform)

train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

In [80]:
# not use dataset
train = pd.read_csv("./2021-ai-midterm-p5/train.csv")
test = pd.read_csv("./2021-ai-midterm-p5/test.csv")
submission = pd.read_csv("./2021-ai-midterm-p5/submit_sample.csv")

In [81]:
# check csv
print(train.info())
print(train.head())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4788 entries, 0 to 4787
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             4788 non-null   int64  
 1   Unnamed: 0        4788 non-null   int64  
 2   customerID        4788 non-null   object 
 3   gender            4788 non-null   object 
 4   SeniorCitizen     4788 non-null   int64  
 5   Partner           4788 non-null   object 
 6   Dependents        4788 non-null   object 
 7   tenure            4788 non-null   int64  
 8   PhoneService      4788 non-null   object 
 9   MultipleLines     4788 non-null   object 
 10  InternetService   4788 non-null   object 
 11  OnlineSecurity    4788 non-null   object 
 12  OnlineBackup      4788 non-null   object 
 13  DeviceProtection  4788 non-null   object 
 14  TechSupport       4788 non-null   object 
 15  StreamingTV       4788 non-null   object 
 16  StreamingMovies   4788 non-null   object 


In [92]:
# preprocessing

# object to int
for col in train.columns:
    if train[col].isnull().sum() != 0 : # 결측값이 있다면
        #최빈값으로 처리
        train[col] = train[col].fillna(train[col].mode()[0]) 
for col in test.columns:
    if test[col].isnull().sum() != 0:
        test[col] = test[col].fillna(test[col].mode()[0])

# object to label
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in test.columns:
    if test[col].dtype == "object" or train[col].dtype =="object":
        test[col] = test[col].astype(str)
        train[col] = train[col].astype(str)
        total = pd.concat([test[col],train[col]],axis=0)
        le.fit(total)
        test[col] = le.transform(test[col])
        train[col] = le.transform(train[col])
train['Churn'] = le.fit_transform(train['Churn'])


# drop unuse column
# train, val  = torch.utils.data.random_split(dataset,[7,3],torch.Generator().manual_seed(42))
train_x = np.array(train.drop(['index','Unnamed: 0','Churn','customerID'],axis=1))
test_x = np.array(test.drop(['index','Unnamed: 0','customerID'],axis=1))
train_y = np.array(train['Churn'])

# scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

# load data on tensor
train_x = torch.Tensor(train_x).to(device)
test_x = torch.Tensor(test_x).to(device)
train_y = torch.Tensor(train_y).to(device)

In [95]:
# check data shape
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(len(torch.unique(train_y)))

torch.Size([4788, 19])
torch.Size([4788])
torch.Size([1198, 19])
2


Init signature:
SimpleImputer(
    *,
    missing_values=nan,
    strategy='mean',
    fill_value=None,
    verbose=0,
    copy=True,
    add_indicator=False,
)
Docstring:     
Imputation transformer for completing missing values.

Read more in the :ref:`User Guide <impute>`.

.. versionadded:: 0.20
   `SimpleImputer` replaces the previous `sklearn.preprocessing.Imputer`
   estimator which is now removed.

Parameters
----------
missing_values : int, float, str, np.nan or None, default=np.nan
    The placeholder for the missing values. All occurrences of
    `missing_values` will be imputed. For pandas' dataframes with
    nullable integer dtypes with missing values, `missing_values`
    should be set to `np.nan`, since `pd.NA` will be converted to `np.nan`.

strategy : string, default='mean'
    The imputation strategy.

    - If "mean", then replace missing values using the mean along
      each column. Can only be used with numeric data.
    - If "median", then replace missing values